In [1]:
# Import librairies
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
# Import datasets
movies = pd.read_csv('ml-1m/movies.dat', sep='::', header=None, engine='python', encoding='latin-1')
users = pd.read_csv('ml-1m/users.dat', sep='::', header=None, engine='python', encoding='latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', header=None, engine='python', encoding='latin-1')

In [3]:
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [5]:
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [6]:
# Training_set & test_set preparation
training_set = pd.read_csv('ml-100k/u1.base', delimiter='\t', header=None)
test_set = pd.read_csv('ml-100k/u1.test', delimiter='\t', header=None)

In [7]:
training_set.head()

,0,1,2,3
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


In [8]:
test_set.head()

,0,1,2,3
0,1,6,5,887431973
1,1,10,3,875693118
2,1,12,5,878542960
3,1,14,5,874965706
4,1,17,3,875073198


In [9]:
# Transform dataframes to arrays
training_set = np.array(training_set, dtype='int')
test_set = np.array(test_set, dtype='int')

In [10]:
training_set[:5]

array([[        1,         1,         5, 874965758],
       [        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       [        1,         4,         3, 876893119],
       [        1,         5,         3, 889751712]])

In [11]:
test_set[:5]

array([[        1,         6,         5, 887431973],
       [        1,        10,         3, 875693118],
       [        1,        12,         5, 878542960],
       [        1,        14,         5, 874965706],
       [        1,        17,         3, 875073198]])

In [12]:
# Get the number of users & films
nb_users = int(max(max(training_set[:, 0]), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1])))

In [13]:
print('Number of users: {}\nNumber of movies: {}'.format(nb_users, nb_movies))

Number of users: 943
Number of movies: 1682


In [14]:
# Convert datas to matrix
def convert(data):
    new_data = []
    for id_users in range(1, nb_users+1):
        id_movies = data[data[:, 0] == id_users, 1]
        id_ratings = data[data[:, 0] == id_users, 2]
        ratings = np.zeros(nb_movies)
        ratings[id_movies-1] = id_ratings
        new_data.append(list(ratings))
    return new_data

In [15]:
training_set = convert(training_set)
test_set = convert(test_set)

In [16]:
training_set[0][0]

5.0

In [17]:
test_set[0][0]

0.0

In [18]:
# Convert datas to tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [19]:
training_set[0]

tensor([5., 3., 4.,  ..., 0., 0., 0.])

In [20]:
test_set[0]

tensor([0., 0., 0.,  ..., 0., 0., 0.])

In [25]:
# Create the neurons network
class SAE(nn.Module):
    
    def __init__(self):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
        
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x

In [26]:
# Instance class
sae = SAE()

In [27]:
# Criterion & optimizer
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr=0.01, weight_decay=0.5)

In [30]:
# Training
nb_epochs = 200

for epoch in range(1, nb_epochs + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies / float(torch.sum(target.data > 0)+1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.item() * mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: {}, Loss: {}'.format(epoch, train_loss/s))

epoch: 1, Loss: 1.7607659557801238
epoch: 2, Loss: 1.096450477990699
epoch: 3, Loss: 1.0532403592614283
epoch: 4, Loss: 1.0382185609567962
epoch: 5, Loss: 1.0310376309171847
epoch: 6, Loss: 1.0266541647937049
epoch: 7, Loss: 1.0239033228538925
epoch: 8, Loss: 1.0218336333070217
epoch: 9, Loss: 1.020944113399351
epoch: 10, Loss: 1.0196726866313186
epoch: 11, Loss: 1.0189782162706758
epoch: 12, Loss: 1.0181537160585732
epoch: 13, Loss: 1.0180544861818686
epoch: 14, Loss: 1.017577125259914
epoch: 15, Loss: 1.0171057047130045
epoch: 16, Loss: 1.0170217125011192
epoch: 17, Loss: 1.0165662463586052
epoch: 18, Loss: 1.0166250542536384
epoch: 19, Loss: 1.0164282518360326
epoch: 20, Loss: 1.0162050068726889
epoch: 21, Loss: 1.0162411627231325
epoch: 22, Loss: 1.0160402007304383
epoch: 23, Loss: 1.0159720278317468
epoch: 24, Loss: 1.0156495975131694
epoch: 25, Loss: 1.0158477197957914
epoch: 26, Loss: 1.0153458747675408
epoch: 27, Loss: 1.0152651116566385
epoch: 28, Loss: 1.0151695295366996
epoc

In [33]:
# Test
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user]).unsqueeze(0)
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies / float(torch.sum(target.data > 0)+1e-10)
        test_loss += np.sqrt(loss.item() * mean_corrector)
        s += 1.
print('Loss: {}'.format(test_loss/s))

Loss: 0.9555766771270394
